## Scenario 1: A single data scientist participating in an ML competition

MLflow setup:
* Tracking server: no
* Backend store: local filesystem
* Artifacts store: local filesystem

The experiments can be explored locally by launching the MLflow UI.

In [1]:
import mlflow

In [6]:
print(mlflow.__version__)
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

2.22.0
tracking URI: 'file:///c:/Users/Damilare/Desktop/DataClubs/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns'


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:///c:/Users/Damilare/Desktop/DataClubs/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/735106529261576010', creation_time=1747948239673, experiment_id='735106529261576010', last_update_time=1747948239673, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='file:///c:/Users/Damilare/Desktop/DataClubs/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/0', creation_time=1747947918734, experiment_id='0', last_update_time=1747947918734, lifecycle_stage='active', name='Default', tags={}>]

### Creating an experiment and logging a new run

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {
        "C": 0.1, 
        "random_state": 42
    }
    
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/05/24 09:38:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'file:///c:/Users/Damilare/Desktop/DataClubs/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/735106529261576010/37e72bedb52c4e9db5c6c1272529d1db/artifacts'


In [8]:
import mlflow

experiments = mlflow.search_experiments()
for exp in experiments:
    print(f"Experiment ID: {exp.experiment_id}, Name: {exp.name}, Artifact Location: {exp.artifact_location}")


Experiment ID: 735106529261576010, Name: my-experiment-1, Artifact Location: file:///c:/Users/Damilare/Desktop/DataClubs/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/735106529261576010
Experiment ID: 0, Name: Default, Artifact Location: file:///c:/Users/Damilare/Desktop/DataClubs/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/mlruns/0


In [ ]:
# mlflow.show_experiments()

### Interacting with the model registry

In [1]:
import mlflow
print(mlflow.__version__)

2.22.0


In [8]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Get the most recent run
experiment = client.get_experiment_by_name("my-experiment-1")  # or your experiment name
runs = client.search_runs([experiment.experiment_id], order_by=["start_time DESC"], max_results=1)

if runs:
    run_id = runs[0].info.run_id
    print(f"Found run ID: {run_id}")
    
    # Now register the model
    model_uri = f"runs:/{run_id}/models"
    registered_model = mlflow.register_model(model_uri, "my-sklearn-model")
    print(f"Model registered: {registered_model.name}")
else:
    print("No runs found")

Found run ID: d2ecd24071054713a725fd451b0fa01e
Model registered: my-sklearn-model


Successfully registered model 'my-sklearn-model'.
Created version '1' of model 'my-sklearn-model'.


In [9]:
from mlflow.tracking import MlflowClient
from mlflow.exceptions import MlflowException

client = MlflowClient()

try:
    models = client.search_registered_models()
    print("✅ Model registry is accessible!")
    print(f"Found {len(models)} registered models")
except MlflowException as e:
    print("❌ It's not possible to access the model registry :(")
    print(f"Error details: {e}")

✅ Model registry is accessible!
Found 1 registered models
